# Aula - Otimização de Hiperparâmetros

Hoje, vamos discutir como selecionar os melhores hiperparâmetros para um dado problema.

1. Introdução à otimização de hiperparâmetros.
2. Usando Grid Search
3. Usando Random Search

# 1. Introdução à otimização de hiperparâmetros

Juntando o que foi visto no curso de estatística e o que foi visto até agora no curso de aprendizado de máquina, já aprendemos alguns modelos.

<font size=4> <b> Classificação </b> </font>
- Regressão Logística
- Árvore de decisão (CART)
- KNN


<font size=4> <b> Regressão </b> </font>
- Regressão Linear 
- Árvore de decisão (CART)
- KNN


Os de regressão logística e regressão linear não têm muitas escolhas prévias que se precisa fazer. No máximo determinar a melhor regularização.

Já o KNN  e as árvores de decisão têm configurações intrínsecas que afetam completamente o resultado, como o número de vizinhos ("K") e a profundidade da árvore, respectivamente. Essas configurações a gente chama de __hiperparâmetros__, e eles controlam o aprendizado.

Queremos então descobrir quais hiperparâmetros vão me ajudar a gerar o melhor modelo possível para o meu problema. Para um ou até dois parâmetros esse processo pode ser feito facilmente "na mão". Porém, à medida que a quantidade de hiperparâmetros aumenta, a quantidade de testes que temos que fazer aumentará __exponencialmente__ e será bem útil fazer uso de algumas técnicas conhecidas.

## 1.1 Diferença entre hiperparâmetro e parâmetro

As nomenclaturas podem parecer um pouco confusas e parecidas a primeira vista, mas as diferenças são perceptíveis ao enxergar com mais atenção. 

O __parâmetro de um modelo é algo que será ajustado no processo de treinamento dele e depende dos dados__. Os parâmetros são parte do modelo e são aprendidos através dos dados. Geralmente os parâmetros são estimados utilizando-se algum algoritimo de otimização.

Já um __hiperparâmetro é uma configuração externa que controla o processo de treinamento__. Os hiperparâmetros não são estimados diretamente pelos dados, como os parâmetros. Geralmente fazemos um tuning para estimá-los.

| Parâmetro | Hiperparâmetro |
|-----------|----------------|
| Configurações internas do modelo | São explicitamente especificadas para controlar o treinamento |
| Essenciais para realizar as predições | São essenciais para otimizar o modelo | 
| Especificadas ou estimadas DURANTE o treinamento | Definidas ANTES do treinamento |
| São internas ao modelo | São externas ao modelo |
| São aprendidas e setadas pelo modelo | Setadas manualmente via tuning |
| Estimados por algoritimos de otimização como Gradiente Descendente | Estimados via tuning dos hiperparâmetros |
| Decidem a performance em dados desconhecidos | Decidem a qualidade do modelo |
| Exemplos: coeficientes da equação em uma Regressão Linear ou Logística, as regras criadas pela Árvore de Decisão, o centróide do cluster | Profundidade da árvore, o K do KNN |


## 1.2. Como otimizar hiperparâmetros?

A forma mais direta de pensar em como fazer essa otimização é assumir que cada escolha de hiperparâmetros é um modelo diferente. Assim, vamos treinar o modelo com cada escolha em um conjunto de treino, e comparar todos com uma estratégia de avaliação de modelos (usando um conjunto de validação ou uma validação cruzada).

Isso é o mesmo que fizemos até agora para avaliação de modelos.

A única diferença é que, como dito antes, a quantidade de escolhas cresce exponencialmente com a quantidade de hiperparâmetros. Se tivermos 2 hiperparâmetros, cada um com 4 valores, teríamos $4^2 = 16 $ escolhas possíveis. Se tivermos 4 hiperparâmetros, teríamos $ 4^4 = 256 $ escolhas possíveis.

Isso nos motiva a criar estratégias quanto a como avaliar todas essas escolhas. Existem 2 estratégias básicas que usamos:
- Grid Search
- Random Search

Vamos exemplificar cada uma dessas estratégias usando um modelo de árvore de decisão.

Vamos começar sem usar nenhuma estratégia, e ver qual seria o nosso baseline.

In [ ]:
# Importando as bibliotecas para matemática e visualização
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
with open('../data/winequality.names', 'r') as fp:
    print(fp.read())

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# Pegando os dados
wine = pd.read_csv('../data/wine_quality_white.csv', sep=';')

In [ ]:
# Separa em treino e teste
X = wine.drop(columns=['quality'])
y = wine['quality']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, shuffle=True, random_state=42)


In [ ]:
X_train.head()

In [ ]:
y_train.value_counts()

Vamos converter nosso problema de multi-classes em um binário

In [ ]:
# Converte y_train para dataframe


# Verifica classes desbalanceadas

In [ ]:
# Instancia o DT
model_sem_otim = 

# Faz a validação cruzada
results_no_optim = 

In [ ]:
results_no_optim

In [ ]:
print(f" Acurácia treino: {100*results_no_optim['train_score'].mean():0.1f}")

acc_sem_otimizacao = results_no_optim['test_score'].mean()
print(f" Acurácia na validação: {100*acc_sem_otimizacao:0.1f}")

In [ ]:
# Faz o fit nos dados de treino


# Faz a predição no teste


print(f" Acurácia teste: {100*accuracy_score(y_test, y_pred):0.1f}")

# 2. Grid Search

É o nosso método extensivo e de força bruta. Escolhemos os valores que queremos testar para nossos hiperparâmetros e testamos todas as escolhas possíveis. Essa estratégia vai ser __MUITO__ custosa computacionalmente e tende a demorar bastante.

<center><img src="https://www.researchgate.net/profile/Karl-Ezra-Pilario/publication/341691661/figure/fig2/AS:896464364507139@1590745168758/Comparison-between-a-grid-search-and-b-random-search-for-hyper-parameter-tuning-The.png" style="height: 350px"/></center>


O `scikit-learn` tem um função que pode nos ajudar nesse processo. Está dentro da parte de `model_selection` e se chama [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV), que utiliza o método de validação cruzada.

Atenção com a escolha da métrica:

<img src="images/model_evaluation.png">

[link](https://scikit-learn.org/stable/modules/model_evaluation.html)

In [ ]:
# Instancia novamente O DT
model_grid = 

In [ ]:
print("Hiperparâmetros do DecisionTreeClassifier:")
model_grid.get_params()

Vamos criar um grid de parâmetros a serem testados:

In [ ]:
X_train.shape

In [ ]:
# Critério do split
criterions = ['gini', 'entropy']

# Profundidades máximas que iremos testar
max_depth = list(np.arange(2, 10))
max_depth.append(None)

# Número de pontos mínimos necessário para permitir um split no nó
min_samples_split = np.arange(4, 11)

# Número de pontos mínimos que podem existir em cada folha (nó final)
min_samples_leaf = np.arange(2, 5)

# Criamos o grid de escolhas
params_grid = {'criterion': criterions,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

params_grid

Total de modelos a serem comparados:

In [ ]:
len(criterions)*len(max_depth)*len(min_samples_split)*len(min_samples_leaf)

In [ ]:
# Importa a classe GridSearchCV do sklearn.model_selection


In [ ]:
# Construindo o objeto "otimizador via grid search com validação cruzada" verbose=2
grid_search = 

In [ ]:
%%time
# Faz o fit do grid search


In [ ]:
# print(grid_search.cv_results_)

In [ ]:
# Converte cv_results_ em pandas dataframe


In [ ]:
# Retorna os melhores parâmetros e o melhor score


In [ ]:
# Obtém o best_score do treino
acc_grid_search_train = df.sort_values("rank_test_score")['mean_train_score'].iloc[0]
print(f" Acurácia treino: {100*acc_grid_search_train:0.5f}")

In [ ]:
# Obtém o best_score da validação
acc_grid_search = 

print(f" Acurácia validação: {100*acc_grid_search:0.5f}")

In [ ]:
# Retorna o best_estimator_


In [ ]:
# Store the best model in a variable to reference later


No final da validação cruzada o grid search, por default, re-treina o modelo utilizando os melhores parâmetros encontrados no dataset inteiro. E retorna para a gente um método `predict` com esse modelo.

For multiple metric evaluation, this needs to be a str denoting the scorer that would be used to find the best parameters for refitting the estimator at the end.

In [ ]:
# Faz o predict no conjunto de teste


print(f" Acurácia teste: {100*accuracy_score(y_test, y_pred_grid):0.1f}")

# 3. Random Search

### [Random Search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)

É parecido com o grid search, pois vamos montar um grupo de escolhas possíveis. Porém, ao invés de compararmos todas as escolhas, nós pegamos uma __amostra aleatória__ (sem reposição) delas, e selecionamos o melhor caso dentro dessa amostra. 

<center><img src="https://www.researchgate.net/profile/Karl-Ezra-Pilario/publication/341691661/figure/fig2/AS:896464364507139@1590745168758/Comparison-between-a-grid-search-and-b-random-search-for-hyper-parameter-tuning-The.png" style="height: 350px"/></center>

In [ ]:
# Instancia DT novamente
model_random = 

# Importa model_selection.RandomizedSearchCV


In [ ]:
# O parâmetro n_iter vai controlar o tamanho da nossa amostra.
random_search = 

In [ ]:
%%time
# Faz o fit do random_search


In [ ]:
# Converte o cv_results_ em pandas dataframe
df = 

In [ ]:
# Retorna melhores parâmetros


In [ ]:
# Obtém o best_score do treino
acc_random_search_train = df.sort_values("rank_test_score")['mean_train_score'].iloc[0]
print(f" Acurácia treino: {100*acc_random_search_train:0.1f}")

# Obtém o best_score da validação
acc_random_search = random_search.best_score_
print(f" Acurácia validação: {100*acc_random_search:0.1f}")

# Faz o predict no X_test
y_pred_random = 

print(f" Acurácia teste: {100*accuracy_score(y_test, y_pred_random):0.1f}")

# Comparando resultados

In [ ]:
print("Comparação das Acurácias: ")
print('Acurácia sem Otimização:         ', np.round(100*acc_sem_otimizacao, 2))
print('Acurácia com GridSearchCV:       ', np.round(100*acc_grid_search, 2))
print('Acurácia com RandomizedSearchCV: ', np.round(100*acc_random_search, 2))

### Avaliando mais de uma métrica
Podemos passar mais de uma métrica para avaliação, mas a escolha de melhor modelo e parâmetros irá depender da métrica indicada no parâmetro `refit`

In [ ]:
#Metrics for Evualation:
met_grid= ['accuracy', 'f1']

random_search_m = RandomizedSearchCV(estimator = model_random, 
                                   param_distributions = params_grid, 
                                   scoring=met_grid,
                                   refit='accuracy',
                                   n_iter=10, 
                                   cv=3, 
                                   verbose=2,
                                   return_train_score=True)

random_search_m.fit(X_train, y_train)
y_pred = random_search_m.predict(X_test)

In [ ]:
df = pd.DataFrame(random_search_m.cv_results_)
df.head()

_____________________________
_____________________________
_____________________________


## Exercícios
1. Faça a otimização de parâmetros para o Random Forest e o KNN das aulas anteriores.

2. Utilize o exemplo abaixo para investigar o comportamento de alguns hiperparâmetros 

In [ ]:
# list of integers 1 to 30
# integers we want to try
hiperparameter_range = range(1, 51)

# list of scores from k_range
train_scores = []
pred_scores = []

# 1. we will loop through reasonable values of hiperparameter
for k in hiperparameter_range:
    # 2. run KNeighborsClassifier with k neighbours
    model = DecisionTreeClassifier(max_depth=k)
    # 3. obtain cross_validate for KNeighborsClassifier with k neighbours
    scores = cross_validate(model, X_train, y_train, cv=3, scoring='accuracy')
    # 4. append mean of scores
    train_scores.append(scores['test_score'].mean())
    # 5. train model
    model.fit(X_train, y_train)
    # 6. predict on test
    y_pred = model.predict(X_test)
    # 7. append accuracy score for predictions
    pred_scores.append(accuracy_score(y_test, y_pred))


In [ ]:
# plot the value of hiperparameter (x-axis) versus the accuracy (y-axis)
plt.plot(hiperparameter_range, train_scores, c='blue', label='Cross-Validated Accuracy')
plt.plot(hiperparameter_range, pred_scores, c='red', label='Prediction Accuracy')

plt.xlabel('Value of hiperparameter')
plt.ylabel('Accuracy')
plt.legend()



## Bibliografia e Aprofundamento
- [Bayesian x Random Search](https://miro.medium.com/max/1400/1*Xfnh-biDrMCECEO37qecKQ.png)
- [Outras otimizações: Evolutionary Search e Gradient Search](https://www.youtube.com/watch?v=TP9W7hmb0Bs)
- [XGBoost Hyperparameter Tuning - A Visual Guide](https://kevinvecmanis.io/machine%20learning/hyperparameter%20tuning/dataviz/python/2019/05/11/XGBoost-Tuning-Visual-Guide.html)
- [Como o tunning afeta o overfiting - bem legal!](https://github.com/tirthajyoti/Machine-Learning-with-Python/blob/master/Complexity_Learning_curves/Complexity_Learning_Analysis_Lending_Data.ipynb)
- [Plot de hiperparâmetros](https://www.ritchieng.com/machine-learning-efficiently-search-tuning-param/)